# Análise Brasileirão

In [1]:
# Pacotes
import pandas as pd 
import scipy.stats 
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode,iplot
from scipy.stats import poisson,skellam, binom
init_notebook_mode(connected=True)# Visualizar no próprio jupyter
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True)
%matplotlib inline

In [2]:
#Lendo a base de dados
os.getcwd()

'D:\\Dropbox\\Projeto Brasileirao\\códigos'

In [3]:
db = pd.read_csv("D:\Dropbox\Projeto Brasileirao\dados\Brasileiro\Base_brasileirao.csv", encoding='latin1', sep= ";")

In [4]:
db.sample(2)

,ano,date,hour,round,home_team,score,away_team,placar_casa,placar_visitante,resultado,arena,state
3850,2016,05/06/2016,11:00,6,América - MG,1 x 0,Figueirense - SC,1.0,0.0,1,Independência - Belo Horizonte - MG,NaN
707,2007,28/10/2007,18:10,33,Vasco,2 x 2,Palmeiras/SP,2.0,2.0,2,São Januário,NaN


### Pré processamento da base

In [5]:
db = db.drop(['date', 'hour', 'arena', 'state'], axis = 1)

In [6]:
def filtro(conditions = [2014, 2015, 2016, 2017, 2018]):
    return db[db['ano'].isin(conditions)]

In [7]:
db = filtro()

In [8]:
db.head(10)

,ano,round,home_team,score,away_team,placar_casa,placar_visitante,resultado
3040,2014,1,São Paulo - SP,3 x 0,Botafogo - RJ,3.0,0.0,1
3041,2014,1,Atlético - PR,1 x 0,Grêmio - RS,1.0,0.0,1
3042,2014,1,Atlético - MG,0 x 0,Corinthians - SP,0.0,0.0,2
3043,2014,1,Bahia - BA,1 x 2,Cruzeiro - MG,1.0,2.0,3
3044,2014,1,Flamengo - RJ,0 x 0,Goiás - GO,0.0,0.0,2
3045,2014,1,Fluminense - RJ,3 x 0,Figueirense - SC,3.0,0.0,1
3046,2014,1,Santos - SP,1 x 1,Sport - PE,1.0,1.0,2
3047,2014,1,Internacional - RS,1 x 0,Vitória - BA,1.0,0.0,1
3048,2014,1,Criciúma - SC,1 x 2,Palmeiras - SP,1.0,2.0,3
3049,2014,1,Chapecoense - SC,0 x 0,Coritiba - PR,0.0,0.0,2


In [9]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1900 entries, 3040 to 4939
Data columns (total 8 columns):
ano                 1900 non-null int64
round               1900 non-null int64
home_team           1900 non-null object
score               1900 non-null object
away_team           1900 non-null object
placar_casa         1899 non-null float64
placar_visitante    1899 non-null float64
resultado           1900 non-null int64
dtypes: float64(2), int64(3), object(3)
memory usage: 133.6+ KB


In [10]:
db.resultado = db.resultado.astype('category')

In [76]:
Gols_casa= go.Box(y=db.placar_casa,
                 name = 'Gols Mandante',
                 marker = dict(
                color = 'rgb(10, 58, 97)' ),
                 boxmean=True,
                 )
Gols_Visitante = go.Box(y=db.placar_visitante,
                       name = 'Gols Visitante',
                        marker = dict(
                        color = ' rgb(233, 104, 59)'
                        ),
                       boxmean=True)
data = [Gols_casa, Gols_Visitante]
layout = go.Layout(
    title = "Box Plot da distribuição de gols marcados no Campeonato Brasileiro 2014 - 2018",
      yaxis=dict(
        dtick=0.5,
        tickcolor='#000',
        showticklabels=True,
        tickangle=0,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        )
    )
)
fig = go.Figure(data=data,layout=layout)

iplot(fig, filename = "Box Plot Styling Outliers")
pio.write_image(fig, 'D:\\Dropbox\\Projeto Brasileirao\\resultados\\Dissertacao\\boxplot_gols_marcados.pdf')


In [128]:
trace1 = go.Bar(
    x=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0 ],
    y=[0.22590837, 0.35439705, 0.25065824, 0.12216956, 0.03159558,
       0.01263823, 0.00263296],
    name='Gols Mandante',
     marker=dict(
        color='rgb(10, 58, 97)',         
    )
)

trace2 = go.Scatter(
    x=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0 ],
    y=[poisson.pmf(i,db.placar_casa.mean()) for i in range(7)],
    name='Poisson({})'.format(round(db.placar_casa.mean(),3)),
     marker=dict(
        color='rgb (20, 137, 255)',
         symbol = 14,
         size= 10,
         
    )
)

trace3 = go.Bar(
    x=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0 ],
    y=[0.41284887, 0.35229068, 0.17219589, 0.04897314, 0.01263823,
       0.00052659],
    name='Gols Visitante',
     marker=dict(
        color='rgb(233, 104, 59)'
    )
)

trace4 = go.Scatter(
    x=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0 ],
    y=[poisson.pmf(i,db.placar_visitante.mean()) for i in range(7)],
    name='Poisson({})'.format(round(db.placar_visitante.mean(),3)),
     marker=dict(
        color=[],
         symbol = 120
    )
)


data = [trace1, trace2, trace3, trace4]
layout = {
  'yaxis': {'title': 'Proporção de gols marcados','autorange':False, 'range':[0,.45],'dtick' : 0.05},
  'xaxis': {'title': 'Gols Marcados', 'autorange':False, 'range': [-.5,6],
        'zeroline' :False,
        'showline':False,
        'ticks' : '',
        'showticklabels' : True
    },
  'title': 'Distribuição de gols marcados no Brasileirão 2014 - 2018'
};
fig = go.Figure(data=data,layout=layout)
iplot(fig)
pio.write_image(fig, 'D:\\Dropbox\\Projeto Brasileirao\\resultados\\Dissertacao\\porporcao_gols_marcados.pdf')

#fig = go.Figure(data=data, layout=layout)
#iplot(fig, filename='angled-text-bar')

In [78]:
a = pd.crosstab(index=db["resultado"],  # Make a crosstab
                              columns=db.ano, normalize='columns').reset_index().drop('resultado',axis=1)
c = ['2014', '2015', '2016', '2017','2018', '2014-2018']
v =[0.5184210526315789,
 0.5263157894736842,
 0.531578947368421,
 0.4394736842105263,
 0.5289473684210526,
 0.508947]
e = [0.24210526315789474,
 0.2394736842105263,
 0.25,
 0.2710526315789474,
 0.29473684210526313,
 0.259474]

d =[0.2394736842105263,
 0.23421052631578948,
 0.21842105263157896,
 0.2894736842105263,
 0.1763157894736842,
 0.231579]


In [82]:
data =[]
trace = go.Bar(x=c,
               y=v,
               name='Propoção de vitórias',
               marker=dict(color = 'rgb(10, 58, 97)'))
trace1 = go.Bar(x=c,
               y=e,
               name='Proporção de empates',
               marker=dict(color = 'rgb(253, 232, 138)'))
trace2 = go.Bar(x=c,
               y=d,
               name='Proporção de derrota',
               marker=dict(color='rgb(233, 104, 59)',))
data = [trace, trace1, trace2]
layout = {
  'yaxis': {'title': 'Proporção','autorange':False, 'range':[0,.55],'dtick' : 0.05},
  'xaxis': {'title': 'Anos', 'type': 'category'},
  'title': 'Proporção de vitórias, empates e derrotas no Brasileirão 2014 - 2018'
}
fig = go.Figure(data=data, layout=layout)
pio.write_image(fig, 'D:\\Dropbox\\Projeto Brasileirao\\resultados\\Dissertacao\\proporcao_resultado_ano.pdf')
iplot(fig, filename='grouped-bar')    

In [103]:
z = np.around(np.array(pd.crosstab(index=db.placar_casa, columns=db.placar_visitante, normalize=True)),3)
fig = ff.create_annotated_heatmap(z,showscale=True,
    reversescale=True, colorscale = [[0, 'rgb(10, 58, 97)'], [1,
            'rgb(253, 232, 138)']])
fig.layout.update({'title': 'Proporção de placares observados'})
fig.layout.xaxis.update(showticklabels=True)
fig.layout.xaxis.update(dict(
        title='Gols visitante',
        side =  'bottom',
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        )))
fig.layout.yaxis.update(dict(
        title='Gols mandante',
        side =  'left',
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        )))
fig.layout.yaxis.update(autorange='reversed',
        showgrid=False,
        zeroline=False,
        showline=False,
        ticks='',
        showticklabels=True)
pio.write_image(fig, 'D:\\Dropbox\\Projeto Brasileirao\\resultados\\Dissertacao\\heatmap.pdf')
iplot(fig, filename='annotated_heatmap',)

### Análise da base

In [ ]:
db.describe()

In [ ]:
# Propoção de vitória time casa, empate e derrota do time da casa
pd.crosstab(index=db["resultado"],  # Make a crosstab
                              columns="Proporção", normalize=True)  

In [ ]:
#Propoção de vitória time casa, empate e derrota do time da casa por ano
pd.crosstab(index=db["resultado"],  # Make a crosstab
                              columns=db.ano, normalize='columns') 

In [ ]:
# Frequência relativa de gols marcados pelos times mandantes no campeonato brasileiro de 2014-2018
pd.crosstab(index=db["placar_casa"],  # Make a crosstab
                              columns="count", normalize=True)      # Name the count column

In [ ]:
# Probabilidade por quantidade de gols do time visitante

mu = 1.5
fig, ax = plt.subplots(1, 1)
x = np.arange(scipy.stats.poisson.ppf(0.01, mu), scipy.stats.poisson.ppf(0.99, mu))
ax.plot(x, scipy.stats.poisson.pmf(x, mu), 'o', ms=5, label='poisson (1.5)', )
plt.ylabel('Probabilidade')
plt.title('Probabilidade por quantidade de gols do time da casa')
plt.xticks(x)
ax.vlines(x, 0, scipy.stats.poisson.pmf(x, mu), colors='black', lw=15, alpha=1, label= 'poisson (1.5)')
ax.vlines(x, 0, [0.225908, 0.354397, 0.250658, 0.122170, 0.031596 ], colors='red', lw=5, alpha=1, label='Frequência observada')
plt.legend(loc="upper left", bbox_to_anchor=[0.52, 1],
           ncol=1, shadow=True, title="Legenda", fancybox=True)
#ax.get_legend().get_title().set_color("red")


In [ ]:
pd.crosstab(index=db["placar_casa"],  # Make a crosstab
                              columns=db['ano'], normalize='columns')      # Name the count column

In [ ]:
# Frequência relativa de gols marcados pelos times visitantes no campeonato brasileiro de 2014-2018
pd.crosstab(index=db["placar_visitante"],  # Make a crosstab
                              columns="count", normalize=True)      # Name the count column

In [ ]:
mu = 0.9
fig, ax = plt.subplots(1, 1)
x = np.arange(scipy.stats.poisson.ppf(0.01, mu), scipy.stats.poisson.ppf(0.99, mu))
ax.plot(x, scipy.stats.poisson.pmf(x, mu), 'o', ms=5, label='poisson (0.9)', )
plt.ylabel('Probabilidade')
plt.title('Probabilidade por quantidade de gols do time visitante')
plt.xticks(x)
ax.vlines(x, 0, scipy.stats.poisson.pmf(x, mu), colors='black', lw=15, alpha=1, label = 'poisson (0.9)')
ax.vlines(x, 0, [0.412849, 0.352291, 0.172196, 0.048973 ], colors='red', lw=5, alpha=1, label='Frequência observada')
plt.legend(loc="upper left", bbox_to_anchor=[0.52, 1],
           ncol=1, shadow=True, title="Legenda", fancybox=True)


In [ ]:
pd.crosstab(index=db["placar_visitante"],  # Make a crosstab
                              columns=db['ano'], normalize='columns')  

In [ ]:
pd.crosstab(index=db.placar_casa, columns=db.placar_visitante, normalize=True)

In [ ]:
b = pd.crosstab(index=db.score, columns="count", normalize=True)

In [ ]:
b.sort_values(by='count',ascending=False).head(10)

In [ ]:
pd.crosstab(index = db.ano, columns=db.placar_casa, normalize='index')

In [ ]:
pd.crosstab(index = db.ano, columns=db.placar_visitante, normalize='index')

In [ ]:
db.corr()

In [ ]:
print ("Média placar casa = ", db.placar_casa.mean(), 'Variância placar casa = ', db.placar_casa.var() )

In [ ]:
print ("Média placar visitante = ", db.placar_visitante.mean(), 'Variância placar visitante = ', db.placar_visitante.var() )

In [ ]:
db[ ['ano','placar_casa', 'placar_visitante']].groupby('ano').describe().reset_index()

In [ ]:
def describeporano(base):
    aux  = list(set(base.ano))
    aux.sort()
    for i in aux:
        base2 = base[base.ano==i]
        print ("Média placar casa em", i, "=",  base2.placar_casa.mean(), 'Variância placar casa', i, "=", base2.placar_casa.var())
        print ("Média placar visitante em " ,i, '=', base2.placar_visitante.mean(), 'Variância placar visitante em ', i, '= ', db.placar_visitante.var() )
        
    
    

In [ ]:
describeporano(db)

In [ ]:
sns.set(style="whitegrid")
ax = sns.barplot( y="placar_casa", data=db, palette="Blues_d")

In [ ]:
db.placar_casa.value_counts(normalize=True).plot(kind = "bar")

In [ ]:
db.placar_visitante.value_counts(normalize=True).plot(kind = "bar")